# License

In [ ]:
# Copyright 2022 Universidad de San Andrés' Authors.

In [ ]:
#@title MIT License
#
# Copyright (c) 2022 Universidad de San Andrés
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Entrená tu primer red neuronal: clasificación MNIST

En esta primera instancia, vamos a declarar o importar las dependencias que necesitamos para comenzar a trabajar con el tutorial.

A través de las palabras reservadas de Python3, como por ejemplo `import`, inicializaremos los siguientes paquetes.

In [ ]:
#@title Pip requirements

!pip install colorama --quiet

In [ ]:
%load_ext tensorboard

import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Luego, verificamos que tengamos correctamente instalada la placa de video, o mejor dicho, la *Graphical Processing Unit* (*GPU*).

In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print('Num GPU available:', len(gpu_devices))

## Descargá el set de datos MNIST

Este tutorial usa el conjunto de datos más famoso conocido como [MNIST](http://yann.lecun.com/exdb/mnist). Éste contiene 60.000 imágenes de números a mano alzada, en escala de grises, y un conjunto de datos de prueba de 10.000 ejemplos. Cada imagen son de baja resolución (28x28 pixels) como se ve aquí:

<table>
  <tr><td align="center">
    <img src="https://images.deepai.org/custom-datasets/images/80c67fa1229744fdae147f18240ab04d/mnist.png"
         alt="MNIST Sprite" width="258">
  </td></tr>
  <tr><td align="center">
    <b>Figura 1.</b> <a href="http://yann.lecun.com/exdb/mnist">Muestras MNIST</a><br/>&nbsp;
  </td></tr>
</table>

MNIST es un conjunto de datos utilizado como el "Hola, Mundo!" de Machine Learning para Visión por Computadoras. MNIST contiene imágenes a mano alzada desde el 0 al 9 de baja resolución.

Existen otro tipo de conjunto de datos basados en este llamado [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), los cuales funcionan como punta pie inicial para verificar que el desarrollo de algoritmos funcionan. Sirven generalemente para hacer pruebas y refinamiento de los algoritmos en cuestión. 

Utilizaremos 60.000 imágenes para entrenar una red neuronal y 10.000 imágenes para analizar la exactitud de la red para clasificar cada una de las imágenes. Se utilizará TensorFlow para acceder directamente al conjunto de datos de manera muy práctica.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

## Inspección de los datos

A través de la API de Tensorflow pudimos obtener las imágenes de entrenamiento y prueba. 

En esta ocasión, vamos a inspeccionar las primeras 10 etiquetas asociadas a las primeras 10 imágenes en el conjunto de entrenamiento.

In [ ]:
train_labels[:10]

A continuación, definimos una lista con los nombres de las clases/etiqueta en formato `string` para visualizar un poco mejor las imágenes.

In [ ]:
class_names = [
    'Cero', 'Uno', 'Dos',
    'Tres', 'Cuatro', 'Cinco',
    'Seis', 'Siete', 'Ocho',
    'Nueve'
]

Uno de los primeros pasos es revisar que los datos se encuentren correctamente cargados. Por eso, en pasos anteriores revisamos algunas etiquetas del conjunto de entrenamiento. 

Ahora corroboraremos el tamaño de ambos conjuntos de datos. Habíamos dicho que serían 60.000 imágenes de entrenamiento y 10.000 de prueba.

Por último, mostraremos una de las imágenes de forma más visual con la biblioteca que importamos más arriba llamada `matplotlib`.

In [ ]:
train_images.shape[0]

In [ ]:
test_images.shape[0]

Observamos que cada uno de los pixels de la imagen puede tener un valor entre 0 y 255. Un paso previo para preparar los datos es aplicar una técnica denominada _normalización_. Es decir, haremos que los valores de los pixels estén entre 0 y 1 únicamente.

In [ ]:
plt.figure()
plt.imshow(train_images[0], cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

La normalización la realizamos en ambos conjuntos de datos dividiendo las listas por 255.

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

plt.figure()
plt.imshow(train_images[0], cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

A partir de lo anterior, mostramos 25 imágenes de entrenamiento con sus respectivas etiquetas.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Arquitectura de la red neuronal

Lo siguiente es definir el modelo que nos permitirá entrenarlo y luego clasificar una nueva imagen automáticamente. Para ello se definen las capas de la red neuronal, cada una de las cuáles cumplen un rol muy importante en la construcción del modelo.

En primer lugar, definimos la capa `Flatten` que permitirá convertir una imagen de 28x28 en un vector o lista de longitud 768. Esta capa luego se conecta con una segunda capa denominada `Dense`, equivalente a una capa donde todas sus neuronas se conectan con la anterior y con la siguiente. Por último, definimos una segunda capa `Dense` con la diferencia que la función de activación en la primera es una "Función ReLU", y la segunda una "Función Softmax".

In [ ]:
# Choose the right number of neurons
neurons_number = 1 # @param {type:"number"}

model = keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(neurons_number, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
print(model.summary())

Por útlimo, queremos definir ciertos parámetros importante a la hora de cómo se entrena una red neuronal. Éstos son el tipo de optimizador, la función de error o _loss_, y las métricas de interés.

Respecto a las métricas, sólo hablaremos de la _accuracy_ o exactitud en este tutorial.

In [ ]:
# Choose your learning rate!
learning_rate = 1  # @param {type:"number"}

model.compile(
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## Entrenamiento

¡Ahora a entrenar!

In [ ]:
import os
import datetime

basedir = '/tmp/mnist/'
logdir = os.path.join(basedir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(logdir, exist_ok=True)

%tensorboard --reload_multifile True --logdir {basedir}

In [ ]:
# Choose the right number of epochs too.
epochs = 1  # @param {type:"number"}
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model.fit(train_images, train_labels, epochs=epochs, callbacks=[tensorboard_callback])

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

## Predicción

Luego del entrenamiento, viene la etapa de la _predicción_. 

A partir de las imágenes de prueba, computaremos las predicciones de cada una de ellas con la función `predict()` de `model`. Le pasaremos la lista de imágenes, y obtendremos las predicciones.

In [ ]:
predictions = model.predict(test_images)

Las predicciones no dicen directamente cuál es la etiqueta o clase que le corresponde a la imagen. Las predicciones son los valores que la capa de neuronas de salida entrega. Cada una de las neuronas de salida, que tienen que ser la misma cantidad de neuronas que de etiquetas, dirá **cúal es la probabilidad de que la imagen en cuestión sea de la clase a la que la neurona corresponde**. Estos valores en la jerga de inteligencia artificial se denominan _logits_ en problemas de clasificación.

In [ ]:
predictions[0]

Aquí pueden ver las predicciones de la red neuronal respecto a la primer imagen de prueba.

Se puede ver que la salida es un vector de 10 posiciones, desde el 0 al 9, donde cada posición corresponde a una etiqueta. Es decir, la posición 3 del vector corresponde a la clase o etiqueta "Three".

Finalmente, en los problemas de clasificación se utiliza una función que determina cuál es la clase a la que la entrada corresponde. En este caso, la entrada es la imágen y la salida va a ser la clase o el número que la red indentificó. 

Para extraer esto, es necesario llamar a una función denominada `argmax` de la biblioteca `numpy`.

In [ ]:
np.argmax(predictions[0])

Bien, hasta el momento hemos computado o inferido las etiquetas de cada una de las imágenes de prueba que la red neuronal NUNCA antes había visto.

El paso siguiente es verificar que la red está en lo correcto, o más específicamente, cuán en lo correcto se encuentra. Es por eso que tenemos las etiquetas asociadas al conjunto de datos de prueba. 

Observemos la etiqueta de prueba de la primer imagen.

In [ ]:
test_labels[0]

In [ ]:
# No modificar este código

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.xticks(range(10), class_names, rotation=45)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.xticks(range(10), class_names, rotation=45)
plt.show()

In [ ]:
# Graficar las primeras X imágenes de prueba, su predicción, y la verdadera etiqueta. 
# Predicciones correctas en azul, incorrectas en rojo.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions, test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions, test_labels)
    plt.xticks(range(10), class_names, rotation=45, fontsize=8)
plt.show()

## Probalo con tus imágenes

Ahora vamos a escribir números del 0 al 9 a mano. Para hacerlo lo más parecido a los datos de entrenamiento, vamos a seguir los siguientes pasos:

1. Agarrar un fibrón de color negro. Lo más cargado posible.
2. En una hoja blanca, escribir números del 0 al 9 (a elección).
3. Con tu celular, o de alguna otra manera, sacarle fotos que tengan una relación aspecto de 1:1. Es decir, que la imágen sea cuadrada en lo posible.
4. Pasar las fotos del celular a la computadora con la que se está trabajando. Pueden pasarla por WhatsApp, Telegram, mail, o cualquier otro medio que se les ocurra.
5. Luego, en el panel de la izquierda de Google Colab, tienen que subir las fotos. Recuerden que las fotos tienen que terminar en `.jpeg`, `.jpg`, o `.png`.
6. Por último, tiene que correr las siguientes celdas. 

In [ ]:
#@title Preprocessing function

import cv2
import colorama
import pandas as pd
import ipywidgets as widgets

from ipywidgets import interact, interactive, fixed, interact_manual

def preprocess(lower, upper, filename, dim=fixed((28,28))):
  img = cv2.imread(filename)

  img = cv2.bitwise_not(img)
  resized = cv2.resize(img,(128,128), interpolation = cv2.INTER_AREA)
  gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
  kernel = np.ones((5,5), np.uint8)
  img = cv2.dilate(gray, kernel, iterations=1)
  cv2.normalize(img, img, lower, upper, cv2.NORM_MINMAX)
  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

  fig, axs = plt.subplots(1, 3)

  ax0, ax1, ax2 = axs[0], axs[1], axs[2]

  ax0.imshow(cv2.threshold(resized,1,255,cv2.THRESH_BINARY)[1], cmap=plt.cm.binary)
  ax0.axis('off')
  ax0.title.set_text('Image 1')
  ax1.imshow(cv2.threshold(resized,254,255,cv2.THRESH_BINARY)[1], cmap=plt.cm.binary)
  ax1.axis('off')
  ax1.title.set_text('Image 2')
  ax2.imshow(resized, cmap=plt.cm.binary)
  ax2.axis('off')
  ax2.title.set_text('Image 3')
  plt.show()

  tensor = tf.convert_to_tensor(resized, dtype=tf.float32)
  prediction = model.predict(tf.expand_dims(tensor, 0))

  df1 = pd.DataFrame({'Number': class_names})
  df2 = pd.DataFrame({'Probability': prediction[0]})
  df = df1.join(df2)
  print(df)
  
  print(colorama.Fore.BLUE + f'\nRESULT: {class_names[np.argmax(prediction)]}')

### ¿Cómo preproceso la imagen?

Al ejecutar la celda de abajo, deben seguir los siguientes pasos:

1. En la caja de texto que dice `filename`, escriban el nombre de la imagen que subieron.
2. Mover el slider `lower`hacia la derecha hasta que la imagen 2 se vea clara.
3. Mover el slider `upper` hacia la izquierda, hasta que el número se vea claro en la imagen, y casi no se vean partes del fondo.

In [ ]:
interact(
    preprocess,
    lower=widgets.IntSlider(min= 255, max=1000, step=10, value=300),
    upper=widgets.IntSlider(min=-850, max=0, step=10, value=-50),
    filename='placeholder.jpg'
);